In [1]:
import sys
import pickle
import random
print("check")
#!{sys.executable} -m pip install opencv
!{sys.executable} -m pip install torch==1.10.2+cu113 torchvision==0.11.3+cu113 torchaudio==0.10.2+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html -q
!{sys.executable} -m pip install torch -q
!{sys.executable} -m pip install torchvision -q
!{sys.executable} -m pip install pytorch_lightning -q
!{sys.executable} -m pip install pytorch-lightning -q

check


In [2]:
import torch
torch.__version__

'1.10.2+cu113'

In [3]:
#!{sys.executable} -m  pip install torch==1.11.0+cu113 torchvision==0.12.0+cu113 -f https://download.pytorch.org/whl/torch_stable.html
#!{sys.executable} -m pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu114


import os
import numpy as np
import torch
import pandas as pd

# import the necessary packages
from torch.nn import Module
from torch.nn import Conv2d
from torch.nn import Linear
from torch.nn import MaxPool2d
from torch.nn import ReLU
from torch.nn import LogSoftmax
from torch import flatten

# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")
# import the necessary packages
#from pyimagesearch.lenet import LeNet
from sklearn.metrics import classification_report
from torch.utils.data import random_split
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch import nn
import matplotlib.pyplot as plt
import numpy as np
import argparse
import torch
import time


In [4]:
class LeNet(Module):
    def __init__(self, numChannels, classes):
        # call the parent constructor
        super(LeNet, self).__init__()
        # initialize first set of CONV => RELU => POOL layers
        self.conv1 = Conv2d(in_channels=numChannels, out_channels=512,
            kernel_size=(5, 5))
        self.relu1 = ReLU()
        self.maxpool1 = MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        # initialize second set of CONV => RELU => POOL layers
        self.conv2 = Conv2d(in_channels=512, out_channels=50,
            kernel_size=(5, 5))
        self.relu2 = ReLU()
        self.maxpool2 = MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        # initialize first (and only) set of FC => RELU layers
        self.fc1 = Linear(in_features=36450, out_features=500)
        self.relu3 = ReLU()
        # initialize our softmax classifier
        self.fc2 = Linear(in_features=500, out_features=classes)
        self.logSoftmax = LogSoftmax(dim=1)
        
        
    def forward(self, x):
        # pass the input through our first set of CONV => RELU =>
        # POOL layers
        #print("0:",x.shape)
        x = self.conv1(x)
        #print("1:",x.shape)
        x = self.relu1(x)
        x = self.maxpool1(x)
        # pass the output from the previous layer through the second
        # set of CONV => RELU => POOL layers
        #print("2:",x.shape)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.maxpool2(x)
        #print("3:",x.shape)
        # flatten the output from the previous layer and pass it
        # through our only set of FC => RELU layers
        x = flatten(x, 1)
        #print("4:",x.shape)
        x = self.fc1(x)
        x = self.relu3(x)
        #print("5:",x.shape)
        # pass the output to our softmax classifier to get our output
        # predictions
        x = self.fc2(x)
        output = self.logSoftmax(x)
        # return the output predictions
        return output

In [5]:
# define model param
numChannels = 3
numClasses = 2
# define training hyperparameters
INIT_LR = 1e-2
BATCH_SIZE = 64
EPOCHS = 10
# define the train and val splits
TEST_SPLIT = 0.1
TRAIN_SPLIT = 0.75
VAL_SPLIT = 1 - TRAIN_SPLIT
# set the device we will be using to train the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [6]:
id_path = 'data/feature_maps/input_matchids.pkl'
label_path = 'data/feature_maps/input_labels.pkl'

In [7]:
with open(id_path, 'rb') as f:
    match_ids = pickle.load(f)

#train val test split with seed 42
random.Random(42).shuffle(match_ids)

test_idx = int(TEST_SPLIT * len(match_ids))-1
train_idx = int(TRAIN_SPLIT * len(match_ids))-1 + test_idx

test_ids = match_ids[:test_idx]
train_ids = match_ids[test_idx:train_idx]
val_ids = match_ids[train_idx:]
match_ids

print(len(match_ids),len(test_ids) + len(train_ids) + len(val_ids)- len(match_ids))

277804 0


In [8]:
from torch.utils.data import Dataset

class MapDataset(Dataset):
    def __init__(self, ids):
         
        with open(label_path, 'rb') as f:
            map_labels = pickle.load(f)
        
        self.labels = map_labels
        self.samples = ids
        
    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        with open(f'data/feature_maps/inputs/{self.samples[idx]}.pkl', 'rb') as f:
            tensor = pickle.load(f)
        
        return tensor, self.labels[self.samples[idx]]





In [9]:
with open(id_path, 'rb') as f:
    match_ids = pickle.load(f)
    
    
trainData = MapDataset(train_ids)
valData = MapDataset(val_ids)
testData = MapDataset(test_ids)


In [10]:
# initialize the train, validation, and test data loaders
trainDataLoader = DataLoader(trainData, shuffle=True,
	batch_size=BATCH_SIZE)
valDataLoader = DataLoader(valData, batch_size=BATCH_SIZE)
testDataLoader = DataLoader(testData, batch_size=BATCH_SIZE)
# calculate steps per epoch for training and validation set
trainSteps = len(trainDataLoader.dataset) // BATCH_SIZE
valSteps = len(valDataLoader.dataset) // BATCH_SIZE

In [11]:
i = 0
for (x,y) in trainDataLoader:
    print(x.shape)
    #print("____")
    #print(y)
    break
i

torch.Size([64, 3, 120, 120])


0

In [12]:
# initialize the LeNet model
print("[INFO] initializing the LeNet model...")
model = LeNet(
	numChannels=numChannels,
	classes=numClasses).to(device)
# initialize our optimizer and loss function
opt = Adam(model.parameters(), lr=INIT_LR)
lossFn = nn.NLLLoss()
# initialize a dictionary to store training history
H = {
	"train_loss": [],
	"train_acc": [],
	"val_loss": [],
	"val_acc": []
}
# measure how long training is going to take
print("[INFO] training the network...")
startTime = time.time()
#print(startTime)

# loop over our epochs
for e in range(0, EPOCHS):
    # set the model in training mode
    model.train()
    # initialize the total training and validation loss
    totalTrainLoss = 0
    totalValLoss = 0
    # initialize the number of correct predictions in the training
    # and validation step
    trainCorrect = 0
    valCorrect = 0
    # loop over the training set
    for (x, y) in trainDataLoader:
        #print(x)
        # send the input to the device
        (x, y) = (x.to(device, dtype=torch.float), y.to(device))
        # perform a forward pass and calculate the training loss
        pred = model(x)
        loss = lossFn(pred, y)
        # zero out the gradients, perform the backpropagation step,
        # and update the weights
        opt.zero_grad()
        loss.backward()
        opt.step()
        # add the loss to the total training loss so far and
        # calculate the number of correct predictions
        totalTrainLoss += loss
        trainCorrect += (pred.argmax(1) == y).type(
            torch.float).sum().item()
        
        # switch off autograd for evaluation
    with torch.no_grad():
        # set the model in evaluation mode
        model.eval()
        # loop over the validation set
        for (x, y) in valDataLoader:
            # send the input to the device
            (x, y) = (x.to(device, dtype=torch.float), y.to(device))
            # make the predictions and calculate the validation loss
            pred = model(x)
            totalValLoss += lossFn(pred, y)
            # calculate the number of correct predictions
            valCorrect += (pred.argmax(1) == y).type(
                torch.float).sum().item()
            
    # calculate the average training and validation loss
    avgTrainLoss = totalTrainLoss / trainSteps
    avgValLoss = totalValLoss / valSteps
    # calculate the training and validation accuracy
    trainCorrect = trainCorrect / len(trainDataLoader.dataset)
    valCorrect = valCorrect / len(valDataLoader.dataset)
    # update our training history
    H["train_loss"].append(avgTrainLoss.cpu().detach().numpy())
    H["train_acc"].append(trainCorrect)
    H["val_loss"].append(avgValLoss.cpu().detach().numpy())
    H["val_acc"].append(valCorrect)
    # print the model training and validation information
    print("[INFO] EPOCH: {}/{}".format(e + 1, EPOCHS))
    print("Train loss: {:.6f}, Train accuracy: {:.4f}".format(
        avgTrainLoss, trainCorrect))
    print("Val loss: {:.6f}, Val accuracy: {:.4f}\n".format(
        avgValLoss, valCorrect))
    
    # finish measuring how long training took
endTime = time.time()
print("[INFO] total time taken to train the model: {:.2f}s".format(
    endTime - startTime))
# we can now evaluate the network on the test set
print("[INFO] evaluating network...")
# turn off autograd for testing evaluation
with torch.no_grad():
    # set the model in evaluation mode
    model.eval()

    # initialize a list to store our predictions
    preds = []
    # loop over the test set
    for (x, y) in testDataLoader:
        # send the input to the device
        x = x.to(device, dtype=torch.float)
        # make the predictions and add them to the list
        pred = model(x)
        preds.extend(pred.argmax(axis=1).cpu().numpy())
# generate a classification report
print(classification_report(testData.targets.cpu().numpy(),
	np.array(preds), target_names=testData.classes))

[INFO] initializing the LeNet model...
[INFO] training the network...
[INFO] EPOCH: 1/10
Train loss: 1.199283, Train accuracy: 0.5000
Val loss: 0.694224, Val accuracy: 0.5036

[INFO] EPOCH: 2/10
Train loss: 0.693743, Train accuracy: 0.4995
Val loss: 0.694251, Val accuracy: 0.5036

[INFO] EPOCH: 3/10
Train loss: 0.693608, Train accuracy: 0.5009
Val loss: 0.694221, Val accuracy: 0.5036

[INFO] EPOCH: 4/10
Train loss: 0.693599, Train accuracy: 0.5019
Val loss: 0.694284, Val accuracy: 0.4964

[INFO] EPOCH: 5/10
Train loss: 0.693633, Train accuracy: 0.5009
Val loss: 0.694192, Val accuracy: 0.5036

[INFO] EPOCH: 6/10
Train loss: 0.693621, Train accuracy: 0.5021
Val loss: 0.694496, Val accuracy: 0.5036

[INFO] EPOCH: 7/10
Train loss: 0.693653, Train accuracy: 0.5002
Val loss: 0.694199, Val accuracy: 0.5036

[INFO] EPOCH: 8/10
Train loss: 0.693655, Train accuracy: 0.5004
Val loss: 0.694391, Val accuracy: 0.4964

[INFO] EPOCH: 9/10
Train loss: 0.693720, Train accuracy: 0.4990
Val loss: 0.694767

AttributeError: 

In [ ]:
# plot the training loss and accuracy
plt.style.use("ggplot")
plt.figure()
plt.plot(H["train_loss"], label="train_loss")
plt.plot(H["val_loss"], label="val_loss")
plt.plot(H["train_acc"], label="train_acc")
plt.plot(H["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig(args["plot"])
# serialize the model to disk
torch.save(model, args["model"])